In [1]:
import numpy as np
from aux_red_multi import generar_ventana, corpus_modificado, palabras_a_indice, indices_a_embeddings, W1
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
x_gen,y1,y2_one_hot =generar_ventana(corpus_modificado, palabras_a_indice, 10, indices_a_embeddings)

In [9]:
entradas = x_gen.shape[1]
salidas = W1.shape[0]

inputs = keras.Input(shape=(entradas,))

x = layers.Dense(200, activation='gelu')(inputs)

x = layers.Dense(64, activation='gelu')(x)

outputs = layers.Dense(salidas, activation='softmax')(x)

multicapa_one_hot = keras.Model(inputs, outputs)

# X: embeddings concatenados o features
# Y: índice de palabra central
multicapa_one_hot.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(1e-3))

multicapa_one_hot.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1300)]            0         
                                                                 
 dense_6 (Dense)             (None, 200)               260200    
                                                                 
 dense_7 (Dense)             (None, 64)                12864     
                                                                 
 dense_8 (Dense)             (None, 7757)              504205    
                                                                 
Total params: 777269 (2.97 MB)
Trainable params: 777269 (2.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

multicapa_one_hot.fit(x_gen, y2_one_hot, 
                epochs=2000,
                batch_size=50,
                shuffle=True,
                validation_split=0.2)

Epoch 1/2000
888/888 [==============================] - 14s 14ms/step - loss: 6.5195 - val_loss: 6.8333
Epoch 2/2000
888/888 [==============================] - 15s 17ms/step - loss: 5.7363 - val_loss: 7.0424
Epoch 3/2000
888/888 [==============================] - 13s 15ms/step - loss: 5.3159 - val_loss: 7.3761
Epoch 4/2000
888/888 [==============================] - 13s 14ms/step - loss: 4.9978 - val_loss: 7.7449
Epoch 5/2000
888/888 [==============================] - 14s 15ms/step - loss: 4.6960 - val_loss: 8.1649
Epoch 6/2000
888/888 [==============================] - 12s 14ms/step - loss: 4.4099 - val_loss: 8.8228
Epoch 7/2000
888/888 [==============================] - 12s 14ms/step - loss: 4.1169 - val_loss: 9.2656
Epoch 8/2000
888/888 [==============================] - 13s 14ms/step - loss: 3.8294 - val_loss: 10.2624
Epoch 9/2000
888/888 [==============================] - 13s 14ms/step - loss: 3.5668 - val_loss: 11.1105
Epoch 10/2000
888/888 [==============================] - 13s 1

KeyboardInterrupt: 

In [ ]:
np.savez('pesos_autoencoder_gatos.npz', *multicapa_one_hot.get_weights())
with np.load('pesos_autoencoder_gatos.npz') as data:
    weights = [data[f'arr_{i}'] for i in range(len(data.files))]
multicapa_one_hot.set_weights(weights)

In [12]:
W1_min, W1_max = W1.min(), W1.max()

x_escalado = 2 * ((x_gen - W1_min) / (W1_max - W1_min)) - 1
y_escalado = 2 * ((y1 - W1_min) / (W1_max - W1_min)) - 1

In [10]:
entradas = x_gen.shape[1]
salidas = W1.shape[1]

inputs = keras.Input(shape=(entradas,))

x = layers.Dense(512, activation='gelu')(inputs)

x = layers.Dense(220, activation='gelu')(x)

outputs = layers.Dense(salidas, activation='tanh')(x)

multicapa_embeddings = keras.Model(inputs, outputs)

# X: embeddings concatenados o features
# Y: índice de palabra central
multicapa_embeddings.compile(loss='mse', optimizer=keras.optimizers.Adam(1e-3))

multicapa_embeddings.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1300)]            0         
                                                                 
 dense_9 (Dense)             (None, 512)               666112    
                                                                 
 dense_10 (Dense)            (None, 220)               112860    
                                                                 
 dense_11 (Dense)            (None, 130)               28730     
                                                                 
Total params: 807702 (3.08 MB)
Trainable params: 807702 (3.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
multicapa_embeddings.fit(x_escalado, y_escalado, 
                epochs=2000,
                batch_size=50,
                shuffle=True,
                validation_split=0.2)

Epoch 1/2000
4717/4717 [==============================] - 74s 15ms/step - loss: 0.0329 - val_loss: 0.0331
Epoch 2/2000
4717/4717 [==============================] - 65s 14ms/step - loss: 0.0327 - val_loss: 0.0331
Epoch 3/2000
4717/4717 [==============================] - 72s 15ms/step - loss: 0.0326 - val_loss: 0.0330
Epoch 4/2000
4717/4717 [==============================] - 65s 14ms/step - loss: 0.0326 - val_loss: 0.0330
Epoch 5/2000
3330/4717 [====================>.........] - ETA: 18s - loss: 0.0327

KeyboardInterrupt: 